In [1]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

# Lab 04 - NLP 2

# Introduction (1 point)

## (1 point) Pick one of the datasets between hate and offensive, and justify your choice. Remember that it is for a commercial application (there is a good and a bad answer).

We choose the offensive dataset because the hate dataset is under the CC BY-NC 4.0 license, which means that we cannot use it for commercial purposes.

In [53]:
import numpy as np
import pandas as pd
from bertopic import BERTopic
from umap import UMAP

ModuleNotFoundError: ignored

In [3]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "offensive")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/11916 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/860 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1324 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/offensive/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print("Split label balence:")
print(f"  Train: {np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])*100:.2f}%")
print(f"  Validation: {np.array(dataset['validation']['label']).sum()/len(dataset['validation']['label'])*100:.2f}%")
print(f"  Test: {np.array(dataset['test']['label']).sum()/len(dataset['test']['label'])*100:.2f}%")

print("\nSplit size:")
print(f"  Train: {len(dataset['train']['label'])} tweets")
print(f"  Validation: {len(dataset['validation']['label'])} tweets")
print(f"  Test: {len(dataset['test']['label'])} tweets")

print("\nExample:")
print(f"  Tweet: '{dataset['train']['text'][0]}'")
print(f"  Label: {dataset['train']['label'][0]}")


Split label balence:
  Train: 33.07%
  Validation: 34.67%
  Test: 27.91%

Split size:
  Train: 11916 tweets
  Validation: 1324 tweets
  Test: 860 tweets

Example:
  Tweet: '@user Bono... who cares. Soon people will understand that they gain nothing from following a phony celebrity. Become a Leader of your people instead or help and support your fellow countrymen.'
  Label: 0


## (1 point) Describe the dataset. Look at the splits, proportion of classes, and see what you can figure out by just looking at the text.

text: a string feature containing the tweet.
label: an int classification label with the following mapping:
- 0: non-offensive
- 1: offensive

They are 3 splits: train, validation and test.
 - train: 11.9k tweets (33% offensive)
 - validation: 1.32k tweets (34% offensive)
 - test: 860 tweets (27% offensive)

After looking at the text, we can see that there are a lot tweets talking about politics, loves and the NFL. We can also see that there are a lot of tweets (almost every tweet) with the '@user' mentionning someone. There are also tweets with emojis and hashtags.

## (3 points) Use BERTopic to extract the topics within the data, and the main topics within each class.

In [ ]:
umap_model = UMAP(random_state=42)

topic_model = BERTopic(umap_model=umap_model, embedding_model="all-MiniLM-L6-v2")

In [ ]:
topics, probs = topic_model.fit_transform(dataset["train"]["text"] + dataset["validation"]["text"])

In [ ]:
topics_per_class = topic_model.topics_per_class(dataset["train"]["text"] + dataset["validation"]["text"], dataset["train"]["label"] + dataset["validation"]["label"])
topic_model.visualize_topics_per_class(topics_per_class)

## (1 point) What do you think about the results? How do you think it could impact a model trained on these data?



In [ ]:
# Bigram model
topic_model = BERTopic(umap_model=umap_model, embedding_model="all-MiniLM-L6-v2", n_gram_range=(1, 2))
topics, probs = topic_model.fit_transform(dataset["train"]["text"] + dataset["validation"]["text"])
topics_per_class = topic_model.topics_per_class(dataset["train"]["text"] + dataset["validation"]["text"], dataset["train"]["label"] + dataset["validation"]["label"])
topic_model.visualize_topics_per_class(topics_per_class, title="<b>Topics per Class Bigram</b>")

# Evaluate a model (6 points)

## (2 points) Evaluate their model on the test split of the dataset you picked, using precision, recall, and F1-score.

In [4]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='offensive'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-offensive.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [8]:
# predict test split
test_text = dataset['test']['text']
test_text = [preprocess(t) for t in test_text]
print(len(test_text))
encoded_input = tokenizer(test_text, return_tensors='tf', padding=True, truncation=True, max_length=256)
output = model(encoded_input)

scores = output[0]
scores = softmax(scores, axis=1)

preds = scores.argmax(axis=1)

860


In [9]:
from datasets import load_metric

# Compute metrics
def compute_metrics(preds, refs):
  metric = load_metric("glue", "mrpc")
  metric1 = load_metric("precision")
  metric2 = load_metric("recall")
  print(metric.compute(predictions=preds, references=refs))
  print(metric1.compute(predictions=preds, references=refs))
  print(metric2.compute(predictions=preds, references=refs))

compute_metrics(preds, dataset['test']['label'])

<ipython-input-9-ae27f7bdd292>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")


{'accuracy': 0.8593023255813953, 'f1': 0.7256235827664399}
{'precision': 0.7960199004975125}
{'recall': 0.6666666666666666}


## (2 points) Look for prediction failures. Extract the top 5 misclassified tweets (highest score in wrong class) for each class and discuss what could be wrong with the model.

In [ ]:
top_5_misclassified = np.array(dataset['test']['text'])
top_5_misclassified = top_5_misclassified

bad_scores = scores[preds != dataset['test']['label']] # Remove good predictions

scores_non = np.array([s[0] for s in bad_scores])
scores_off = np.array([s[1] for s in bad_scores])

sorted_indices = scores_non.argsort() # Get the pred with max and get the order of max
top_5_misclassified_non = np.array(top_5_misclassified)[sorted_indices] # sort the text dataset
top_5_misclassified_non = top_5_misclassified_non[:5]

print("Non offensive misclassified")
for misclassified in top_5_misclassified_non:
  print(misclassified)


sorted_indices = scores_off.argsort() # Get the pred with max and get the order of max
top_5_misclassified_off = np.array(top_5_misclassified)[sorted_indices] # sort the text dataset
top_5_misclassified_off = top_5_misclassified_off[:5]

print("Offensive misclassified")
for misclassified in top_5_misclassified_off:
  print(misclassified)

Non offensive misclassified
#NewVideo 🍆🍑💦 Sex Toys, Anyone?
#Twitter. Stop what you are doing right now and head over to #Twitch and listen to my AMAZING friend Grendy as he plays #HatofulBoyfriend. #VoiceActor in the making! #supportsmallstreamers
2018 not surprised it starts from the top!!!  Karma a bitch watch!!!!
#MonbebeSelcaDay she is beautiful
#HurricaneFlorence ... not good, she is sporting a pinhole eye while strengthening...
Offensive misclassified
#Spending. Don't speed all of your money. If you are not saving atlest 15%  of what you earn,  start today.  Look at your income and expenses, find out where you can start to cut back and get 15% you save.
#SilsilaBadallteRishtonKa tag is filled with appreciation of @user only....   Never watched this serial but I always wanted appreciation for Aditi... She is underrated since starting of the show... She is worth of all this appreciation.. 👏👏👏
@user She is beyond famous. She is Lalisa Manoban 😎
#parikshitsolutions Security incident

In [44]:
import json

with open('tweets.json', 'r') as f:
  tweets = json.load(f)

tweets = np.array([preprocess(t['text']) for t in tweets])

In [34]:
scores = []

for tweet in tweets:
  encoded_input = tokenizer(tweet, return_tensors='tf')
  output = model(**encoded_input)
  score = output[0][0].numpy()
  score = softmax(score)
  scores.append(score)

scores = np.array(scores)
preds = scores.argmax(axis=1)

## (2 points) Extract the top 10 tweets your model is most confident about in the target class (offensive or hateful), the top 10 in the neutral class, and the top 10 your model is most uncertain about. Do you believe the model is doing a great job?

In [13]:
diff = np.abs(np.diff(scores).flatten())
diff

array([0.747544  , 0.6743665 , 0.78431326, ..., 0.09780452, 0.8799035 ,
       0.77852356], dtype=float32)

In [23]:
indices = np.argsort(diff)
top_10_neutral = []
for i in indices[:10]:
  top_10_neutral.append(tweets[i])
top_10_neutral

["I'm lucky not literate 🙃",
 'THE PAIN IN MY ARM WORTHS SO BAD',
 'its crazy to me that people think fictional characters could actually have feelings? 💀💀 what is this? shifting tiktok ?',
 'ok I’m gonna cry now',
 'someone pinch me i feel like i’m dreaming',
 'MY PHONES DYING',
 'Why didn’t anyone tell me one piece is also sad af 😒',
 'It’s just become a case of ‘this doctor said this and that doctor said that’, across the board, and not just with Dr Young.\n\nThen, all of the commentators just want to tell us how confused we all are, and so they feed us more stats to prove we’re being silly.\n\n#covid19qld',
 'this is a big brain moment',
 'This is just furry and 100 gecs']

## (1 point) Extract about 100 tweets containing at least 20% of your target class (offensive/hateful), from the 10K tweets provided. You can use the pretrained model to help you find tweets in the target class.

In [51]:
from random import sample
ratio = 0.5
tweets_100 = sample(list(tweets[preds > 0]), int(100 * ratio))
tweets_100.extend(sample(list(tweets[preds < 1]), int(100 * (ratio/1))))
len(tweets_100)

100

In [55]:
# Generate csv
df_tweets = pd.DataFrame(tweets_100)
df_tweets.to_csv("annotation.csv", index=False)